## Palavras Mais Frequentes RHS

1. Obter os dados das postagens da rede Humaniza SUS
2. Tratar o texto obtido
3. Fazer a contagem das palavras mais frequentes por dia
4. Criar um arquivo com essas palavras

In [ ]:
import pymysql
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from collections import Counter

In [ ]:
conn = pymysql.connect(host='localhost',
                           port=3306,
                           user='root',
                           passwd='',
                           db='rhs')
cur = conn.cursor()
cur.execute("SELECT FROM_UNIXTIME(A.created, '%Y') AS Data, C.type AS Tipo, B.comment_body_value AS Comentario FROM comment A INNER JOIN field_data_comment_body B ON A.cid = B.entity_id JOIN node C ON A.nid = C.nid WHERE C.type LIKE 'blog'")

In [ ]:
# armazena o nome de cada uma das colunas do banco de dados
row_headers=[x[0] for x in cur.description]

In [ ]:
# cria uma lista com os dados do banco
rhs_data = []
for row in cur:
    rhs_data.append(dict(zip(row_headers,row)))

In [ ]:
# transforma a list em um dataframe
df = pd.DataFrame(rhs_data)

In [ ]:
# usa o próprio pandas para agrupar os dados por dia/data
df_new = pd.DataFrame(df.groupby('Data')['post'].apply(' '.join))

In [ ]:
# Copia as datas do groupby do index para uma coluna
df_new['Data'] = df_new.index

In [ ]:
# Reseta o index para ficar númerado de acordo com a ordem dos items
df_new = df_new.reset_index(drop=True)

In [ ]:
# Transforma as Datas em Strings (facilita para trabalhar)
df_new['Data'] = df_new['Data'].apply(lambda x: dt.datetime.strftime(x, '%d/%m/%Y'))

In [ ]:
# cria uma lista com as postagens que estão dentro do dataframe
# cria uma nova lista onde serão tratados caracteres em ASCII e volta a inserir o texto tratado dentro do dataframe
dflist = df_new['post'].tolist()

# Limpa erros de códificação das postagens (uma vez que tem erros de ASCII não reconhece caracteres como "ç")
df_html =[]
for item in dflist:
       parser = HTMLParser()
       df_html.append(parser.unescape(item))
    
# Insere novamente o texto dentro do dataframe
df_new['post'] = df_html

A parte de código mostrada abaixo que contém o BeautifulSoup é muito importante, uma vez que foram identificadas muitas postagens com html, em algumas com muita repetição de tags em html

In [ ]:
# usa o beautifulsoup para limpar todo o html das postagens
def cleanMe(html):
    soup = BeautifulSoup(html, "lxml") # cria um novo objeto do bs4 para o item html
    for script in soup(["script", "style"]): # remove todos scripts e css
        script.extract()
    # get text
    text = soup.get_text()
    # quebrar em linhas e remove espaço à esquerda e à direita em cada
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # remove linhas em branco
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [ ]:
# Gera uma lista que armazena o texto tratado pela função feita acima
df_soup = []
for text in df_new.get('post'):
       df_soup.append(cleanMe(text))

# Insere o texto tratado novamente no dataframe
df_new['post'] = df_soup

In [ ]:
# Tratamento simples de texto, remove caracteres especiais e númericos
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"[\,\$\&\*\%\(\)\~\-\"\^\+\#\/|0-9]", " ")
    df[text_field] = df[text_field].str.replace(r"[\.\=\'\:\;\?\!\_\...]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.replace(r"\[[^]]*\]", " ")
    df[text_field] = df[text_field].str.lower()
    return df

# ativa a função acima que percorre o campo do dataframe com as postagens
df_new = standardize_text(df_new, 'post')

In [ ]:
# Remover múltiplos espaços em branco tornando o texto em algo único
df_aux = []
for word in df_new.get('post'):
       df_aux.append(word.split())

# Juntar novamente as palavras de cada um dos post para o texto ficar uniforme
space_correction = [ ' '.join(l) for l in df_aux]

df_new['post'] = final_correction

In [ ]:
# remove stopwords
# Por serem textos mais completos usar um arquivo próprio para stopwords
pathStopwords = "stop_words.txt"

# Read in and split the stopwords file.
with open(pathStopwords, 'r') as f:
    stop_words = f.read().split("\n")

# transforma as postagens que estão no dataframe em lista para retirar as stopwords
dflist = df_new['post'].tolist()

my_new_list = [[word for word in text.split() if word not in stop_words] for text in dflist]
my_new_list = [ ' '.join(l) for l in my_new_list]
df_new['post'] = my_new_list

In [ ]:
# Cria uma lista com as palavras separadas entre si
wordlist = []
for wordlists in df_new.get('post'):
    wordlist.append(wordlists.split())

In [ ]:
# Cria uma lista com a frequência das palavras, armazenando elas de acordo com o dia da postagem
wordfreq = []
c = 0
for c in range(len(wordlist)):
    if c <= 3228:
        wordfreq.append(Counter(wordlist[c]))

In [ ]:
# Cria uma lista com as 10 palavras mais frequentes
wordfreqmost = []
for counter in wordfreq:
    wordfreqmost.append(Counter(counter).most_common(10))

Nessa etapa o output que recebemos é uma lista de tuplas, quando transformamos essa lista em dataframe ela gera uma coluna para cada palavra e o respectivo número de aparições, como verificamos as 10 mais frequentes, são geradas 10 colunas no dataframe, onde renomeamos as colunas com letras

In [ ]:
# Cria um dataframe com as palavras que foram geradas na lista anterior
df_words = pd.DataFrame(wordfreqmost)
df_words.columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [ ]:
# Concatenamos todas as colunas para que as palavras fiquem juntas
d = list(zip(df_words.A,df_words.B,df_words.C,df_words.D,df_words.E,df_words.F,
             df_words.G,df_words.H,df_words.I,df_words.J))

In [ ]:
# Cria um dataframe final vazio, insere a Data da postagem nesse novo dataframe
# e a coluna que contem todas as palavras com o número de aparições juntas
df_final = pd.DataFrame()
df_final['Data'] = df_new['Data']
df_final['Concatenado'] = pd.Series(d)

In [ ]:
# Criamos o arquivo CSV final que será tratado para que os dados sejam colocados no Tableau
df_final.to_csv("rhs_word_cloud.csv", encoding='UTF-8', index=False)